In [ ]:
## https://github.com/vizsim/mapillary_coverage/issues/1

In [42]:
import pandas as pd
import geopandas as gpd
# Read the Parquet files


#gdf_mp_pano = pd.read_parquet("germany_osm-highways_25-04-05_mp_pano_coverage_23-01-01until25-04-01_ratio.parquet")
#gdf_mp_all = pd.read_parquet("germany_osm-highways_25-04-05_mp_all_coverage_23-01-01until25-04-01_ratio.parquet")

gdf_mp_pano = pd.read_parquet("germany_osm-highways_25-05-09_mp_pano_coverage_25-05-10_ratio.parquet")
gdf_mp_regular = pd.read_parquet("germany_osm-highways_25-05-09_mp_regular_coverage_25-05-10_ratio.parquet")


In [43]:
gdf_mp_pano=gdf_mp_pano[["osm_id","mp_coverage_ratio"]].copy()

In [44]:
gdf_mp_pano_above_06=gdf_mp_pano[gdf_mp_pano["mp_coverage_ratio"]>=0.6].copy()

In [45]:
gdf_mp_pano_above_06["mapillary_coverage"] = "pano"

In [46]:
gdf_mp_pano_above_06

,osm_id,mp_coverage_ratio,mapillary_coverage
0,1978,1.00000,pano
4,2293021,1.00000,pano
9,2483295,0.65936,pano
10,2483296,1.00000,pano
17,2597457,0.97025,pano
...,...,...,...
527061,1385144772,1.00000,pano
527062,1385144773,1.00000,pano
527066,1385149711,1.00000,pano
527067,1385149712,1.00000,pano


In [47]:
gdf_mp_regular=gdf_mp_regular[["osm_id","mp_coverage_ratio"]].copy()

In [48]:
gdf_mp_regular_above_06=gdf_mp_regular[gdf_mp_regular["mp_coverage_ratio"]>=0.6].copy()

In [49]:
gdf_mp_regular_above_06

,osm_id,mp_coverage_ratio
2,104,1.000000
6,111,1.000000
7,117,1.000000
8,122,1.000000
9,123,0.874366
...,...,...
2507132,1385153162,0.993251
2507135,1385154522,1.000000
2507136,1385154590,0.868189
2507137,1385157322,1.000000


In [50]:
gdf_mp_regular_above_06["mapillary_coverage"] = "regular"

In [51]:
both_concat=pd.concat([gdf_mp_pano_above_06,gdf_mp_regular_above_06],ignore_index=True)

In [52]:
both_concat["mapillary_coverage"].value_counts()

mapillary_coverage
regular    1353914
pano        295806
Name: count, dtype: int64

In [30]:
both_concat["mapillary_coverage"].value_counts()

mapillary_coverage
regular    1476231
pano        265933
Name: count, dtype: int64

In [53]:
## drop duplicates, keep pano
both_concat = both_concat.sort_values(by="mapillary_coverage", ascending=True).drop_duplicates(subset="osm_id", keep="first")

In [54]:
both_concat["mapillary_coverage"].value_counts()

mapillary_coverage
regular    1241698
pano        295806
Name: count, dtype: int64

In [55]:
both_concat["osm_id"].value_counts()

osm_id
177643587     1
177636568     1
177636567     1
177632296     1
177629008     1
             ..
1070048986    1
1070048984    1
1070048983    1
1070048982    1
1070048990    1
Name: count, Length: 1537504, dtype: int64

In [56]:
both_concat=both_concat[["osm_id","mapillary_coverage"]].copy()

In [57]:
both_concat

,osm_id,mapillary_coverage
0,1978,pano
197207,1070048989,pano
197206,1070048988,pano
197205,1070048986,pano
197204,1070048984,pano
...,...,...
747108,177647549,regular
747107,177647275,regular
747106,177645545,regular
747104,177645063,regular


In [58]:
both_concat.to_csv("output/germany_osm-highways_25-05-09_mp_coverage_23-01-01until25-05-10_ratio_above_06.csv",index=False)

In [ ]:
#